In [1]:
pip install transformers torch scikit-learn flask


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from flask import Flask, request, jsonify
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
import joblib


In [3]:
# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels: inquiry, appointment, symptom
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/mayank/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Example symptom assessment dataset (simplified)
data = {
    'text': ["I have a headache and fever", "My throat is sore", "I feel dizzy and weak"],
    'label': [0, 1, 2]  # 0: General, 1: ENT, 2: Neurology
}

# Convert to DataFrame
import pandas as pd
df = pd.DataFrame(data)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])
y = df['label']

# Train a logistic regression model
symptom_model = LogisticRegression()
symptom_model.fit(X, y)

# Save the model for later use
joblib.dump(symptom_model, 'symptom_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')


['vectorizer.pkl']

In [5]:
app = Flask(__name__)

# Load the trained symptom model and vectorizer
symptom_model = joblib.load('symptom_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')

@app.route('/chatbot', methods=['POST'])
def chatbot():
    user_input = request.json['message']
    
    # Use the NLP model to classify the type of query
    inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    
    response = ""
    
    if predicted_class == 0:  # Inquiry
        response = "How can I assist you with your healthcare needs?"
    elif predicted_class == 1:  # Appointment
        response = "Sure, let's schedule an appointment. Please provide your preferred date and time."
    elif predicted_class == 2:  # Symptom Assessment
        # Use the symptom model to predict possible conditions
        symptom_vector = vectorizer.transform([user_input])
        symptom_prediction = symptom_model.predict(symptom_vector)[0]
        
        if symptom_prediction == 0:
            response = "It seems like a general health issue. Please consult your primary care physician."
        elif symptom_prediction == 1:
            response = "It might be an ENT issue. Would you like to see a specialist?"
        elif symptom_prediction == 2:
            response = "It could be related to neurology. I recommend seeing a neurologist."
    
    return jsonify({"response": response})

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (inotify)
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/mayank/.local/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/mayank/.local/lib/python3.11/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/mayank/.local/lib/python3.11/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mayank/.local/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/mayank/.local/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, 

SystemExit: 1

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
pip install Flask


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from flask import Flask, request, jsonify
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
import joblib
import pandas as pd

# Step 1: Load Pre-trained NLP Model
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels: inquiry, appointment, symptom
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 2: Train a Symptom Assessment Model (for demonstration purposes)
# Example symptom assessment dataset
data = {
    'text': ["I have a headache and fever", "My throat is sore", "I feel dizzy and weak"],
    'label': [0, 1, 2]  # 0: General, 1: ENT, 2: Neurology
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])
y = df['label']

# Train a logistic regression model
symptom_model = LogisticRegression()
symptom_model.fit(X, y)

# Save the model and vectorizer for later use
joblib.dump(symptom_model, 'symptom_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

# Step 3: Create a Flask App for the Chatbot
app = Flask(__name__)

# Load the trained symptom model and vectorizer
symptom_model = joblib.load('symptom_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')

@app.route('/chatbot', methods=['POST'])
def chatbot():
    user_input = request.json['message']
    
    # Use the NLP model to classify the type of query
    inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    
    response = ""
    
    if predicted_class == 0:  # Inquiry
        response = "How can I assist you with your healthcare needs?"
    elif predicted_class == 1:  # Appointment
        response = "Sure, let's schedule an appointment. Please provide your preferred date and time."
    elif predicted_class == 2:  # Symptom Assessment
        # Use the symptom model to predict possible conditions
        symptom_vector = vectorizer.transform([user_input])
        symptom_prediction = symptom_model.predict(symptom_vector)[0]
        
        if symptom_prediction == 0:
            response = "It seems like a general health issue. Please consult your primary care physician."
        elif symptom_prediction == 1:
            response = "It might be an ENT issue. Would you like to see a specialist?"
        elif symptom_prediction == 2:
            response = "It could be related to neurology. I recommend seeing a neurologist."
    
    return jsonify({"response": response})

# Run the Flask app
if __name__ == "__main__":
    app.run(debug=True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 * Serving Flask app '__main__'
 * Debug mode: on


/home/mayank/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (inotify)
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/mayank/.local/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/mayank/.local/lib/python3.11/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/mayank/.local/lib/python3.11/site-packages/traitlets/config/application.py

SystemExit: 1

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
